In [30]:
import random
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Iterable, Optional
import pandas as pd
import numpy as np
from utils import flatten_list

In [31]:
@dataclass
class TeamSpec:
    # 用声明式“配方”定义每队所需角色，以及如何兜底
    # items 是按顺序的需求：("奶", 1) 表示抽取1人奶；("近战", 3) 表示抽3个近战
    main_character: Tuple[str, int]
    fallback: Optional[str] = None  # 可选：当主要类型空时，用次选类型补齐

def pop_random(lst: List, *, rng: random.Random) -> Optional[str]:
    if not lst:
        return None
    i = rng.randrange(len(lst))
    lst[i], lst[-1] = lst[-1], lst[i]
    return lst.pop()

def pop_order(lst: List) -> Optional[str]:
    if not lst:
        return None
    lst[0], lst[-1] = lst[-1], lst[0]
    return lst.pop()

def form_team(spec: TeamSpec, buckets: Dict[str, List[str]], rng: random.Random, random: bool=False) -> Tuple[List[str], List[str], bool]:
    team_ids: List[str] = []
    team_jobs_type: List[str] = []

    def take(role: str, count: int) -> int:
        taken = 0
        for _ in range(count):
            if random:
                pid = pop_random(buckets.get(role, []), rng)
            else:
                pid = pop_order(buckets.get(role, []))

            if pid is None:
                break
            team_ids.append(pid)
            taken += 1
        return taken

    # 先按主需求取
    role = spec.main_character[0]
    n = spec.main_character[1]
    got = take(role, n)
    team_jobs_type.append([role]*got)

    # 不够的从备选职业取
    if got < n and spec.fallback:
        fb_role = spec.fallback
        fb_got = take(fb_role, n-got)
        got += fb_got
        team_jobs_type.append([fb_role]*fb_got)

    # check number of member
    if got != n:
        full = False
    else:
        full = True

    return team_ids, team_jobs_type, full

In [32]:
import copy
from run import main

csv_name = "temp.csv"
today_map, report = main(csv_name)

random_seed = 2025
rng = random.Random(random_seed)
num_member = 6

# 1) 第一队:      奶1 + 火1 + 拳1 + 圣骑1 + 饺子1 + 需要拳的职业(弩，船)
# 2) 第二队(远程): 奶1 + 眼1 + (优先远程4, 不够用眼补齐至4)
# 3) 第三队(洗澡): 奶1 + 火1 + 刀
# 4) 第四队: CSV 中 today_map 但未被前3队使用的(含未知/未映射)

team_flatten = []
job_flatten = []
roles_all = [["奶", "火", "拳", "圣骑", "饺子", ["船", "弩"]],
             ["奶", "火", "弓", ["标", "弓"]],
             ["奶", "火", ["刀", "饺子"]],
            ]
numbers_all = [[1, 1, 1, 1, 1, 1],
               [1, 1, 1, 3],
               [1, 1, 4]
            ]
buckets = copy.deepcopy(today_map)
for i, (roles, numbers) in enumerate(zip(roles_all, numbers_all)):
    team_i, job_i = [], []
    for role,number in zip(roles, numbers):
        if isinstance(role, list):
            spec = TeamSpec(main_character=(role[0], number), fallback=role[1])
        else:
            spec = TeamSpec(main_character=(role, number))
        team_ids, team_jobs_type, full = form_team(spec, buckets, rng)
        if team_ids:
            team_i.append(team_ids)
            job_i.append(team_jobs_type)
        if not full:
            report.add_warning(f"Team{i+1}缺{role}")

    if len(flatten_list(team_i)) < num_member:
        report.add_warning(f"Team{i+1}人不满")

    team_flatten.append(flatten_list(team_i))
    job_flatten.append(flatten_list(job_i))

for warning in report.warnings:
    print("Warnings:", warning)


Warnings: ["duplicated matching in Repo: 38. Flash936—刀 ['Flash936', 'as']", "duplicated matching in Repo: 48. HACK特龙 打火机 ['HACK特龙', 'K']"]
Errors: []
Unmapped: ['43. 江海不洗澡 弓']
Invalid: [('38. Flash936—刀', ['Flash936', 'as']), ('48. HACK特龙 打火机', ['HACK特龙', 'K'])]
Leftover: {}
Warnings: duplicated matching in Repo: 38. Flash936—刀 ['Flash936', 'as']
Warnings: duplicated matching in Repo: 48. HACK特龙 打火机 ['HACK特龙', 'K']


In [33]:
import string
from datetime import datetime
from openpyxl import load_workbook
letters = string.ascii_uppercase

# 1) 载入模板
wb = load_workbook("一条排班.xlsx")
ws = wb["Sheet1"]  # 替换为你的工作表名

# 2) 写入文件
now = datetime.now()

for i, (job, id_list) in enumerate(zip(job_flatten, team_flatten)):
    for j, job_i in enumerate(job):

        ws[f'{letters[i*2]}{3+j}'].value = id_list[j]
        ws[f'{letters[i*2+1]}{3+j}'].value = job_i

wb.save(f"{now.strftime('%Y%m%d')}一条排班.xlsx")

In [34]:
team_flatten = []
job_flatten = []
roles_all = [["奶", "火", "拳", "圣骑", "饺子", ["弩", "船"]],
             ["奶", "火", "弓", ["标", "弓"]],
             ["奶", "火", ["刀", "饺子"]],
            ]
numbers_all = [[1, 1, 1, 1, 1, 1],
               [1, 1, 1, 3],
               [1, 1, 4]
            ]

for i, (roles, numbers) in enumerate(zip(roles_all, numbers_all)):
    team_i, job_i = [], []
    for role,number in zip(roles, numbers):
        if isinstance(role, list):
            spec = TeamSpec(main_character=(role[0], number), fallback=role[1])
        else:
            spec = TeamSpec(main_character=(role, number))
        team_ids, team_jobs_type, full = form_team(spec, buckets, rng)
        if team_ids:
            team_i.append(team_ids)
            job_i.append(team_jobs_type)
        if not full:
            report.add_warning(f"Team{i+1}缺{role}")

    if len(flatten_list(team_i)) < num_member:
        report.add_warning(f"Team{i+1}人不满")

    team_flatten.append(flatten_list(team_i))
    job_flatten.append(flatten_list(job_i))

for warning in report.warnings:
    print("Warnings:", warning)

Warnings: duplicated matching in Repo: 38. Flash936—刀 ['Flash936', 'as']
Warnings: duplicated matching in Repo: 48. HACK特龙 打火机 ['HACK特龙', 'K']


In [35]:
# 2) 写入文件
now = datetime.now()

for i, (job, id_list) in enumerate(zip(job_flatten, team_flatten)):
    for j, job_i in enumerate(job):

        ws[f'{letters[i*2]}{12+j}'].value = id_list[j]
        ws[f'{letters[i*2+1]}{12+j}'].value = job_i

wb.save(f"{now.strftime('%Y%m%d')}一条排班.xlsx")

In [36]:
buckets

{'未知': ['43. 江海不洗澡 弓'],
 '刀': ['龙卷枫', '滴滴叭叭'],
 '单挂': ['3. 羽寒 眼 单挂',
  '14. 星暖 奶 单挂',
  '17. 小水果刀 单挂',
  '20. 南楼-刀 单挂，请把这个号和放在同一个团',
  '26. 阿光军（接村内打卡滴滴代跳）  单挂',
  '39. K 刀 152 单挂',
  '40. 拳 拾光倒流 单挂',
  '42. 真武大帝-147弓 单挂',
  '45. 冰封de记忆 130冰雷 单挂',
  '46. 小纪 153火 单挂',
  '50. 喃 单挂',
  '52. 芙蕾卡—主教 132 6021hp，来单挂',
  '54. 老魔 135标飞 单挂一条',
  '58. 猫猫爱金币 147火毒 （单挂）',
  '60. 呆妹小册那 火 单挂'],
 '圣骑': ['一天也'],
 '奶': [],
 '弓': ['芙啦芙啦', '细语温柔', '库里'],
 '弩': ['桥本奈奈未'],
 '拳': [],
 '标': [],
 '火': [],
 '船': [],
 '饺子': ['厉飞羽']}